In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import date 

#### endFrame: all CAPS in one df

In [3]:
import os

#https://stackoverflow.com/a/21232849 model 
def getCAPSByDateAndType(type):  #returns a dict, date + df caps for that date, then extended date and time
                                # print("for type, ", type)
    ret = []
    for root, dirs, files in os.walk("./lob_caps/"):
        for filename in files:
            if type in filename:
                # print("CAPS file, ", filename)
                ret.append(filename)
    return ret

csvFileList = getCAPSByDateAndType("CAPS") #iterate this array to dip into each csv, later on
li = []                         #form the endFrame / global data frame around this array
for filename in csvFileList:
    csv = "lob_caps/" + filename
    # print(csv)
    df = pd.read_csv(csv, index_col=None, header=0)
    li.append(df)

endFrame = pd.concat(li, axis=0, ignore_index=True) #end frame contains all data
endFrame.sort_values(by=['time'], ascending=True)   #sorted by time into one time series
print("for new df: ", endFrame.shape[0])
start = endFrame["time"].min()
end = endFrame["time"].max()
print("start: ", start, " end: ", end)

for new df:  111336
start:  1660221600292  end:  1671726692823


In [4]:
def getIndicators():
    return pd.read_csv("usIndicatorsFall22-fulls.csv")

In [17]:
from dateutil import parser
import datetime
import time

def convertIndicatorDateToEpoch(original):
    # print("original ", original) #Thu Aug 11 4:30 AM 2022
    re =time.strptime(original, "%a %b %d %I:%M %p %Y")
    ep = time.mktime(re)
    # print("TS ", int(ep))   #1660311116579
    return int(ep)

In [22]:
def getFullIADates():
    indicatorData = getIndicators() #indicators file read, access all indicator data per event/announcement
    result = []
    for row, val in indicatorData.iterrows(): #all indicators
        s1 = val['date'].replace(','  , '')
        ss = s1.split(" ")
        day  = ss[0][0:3]
        mon = ss[1][0:3]
        num = ss[2]
        hr = val['time']
        if int(num) <10:
            num = "0"+num
        yr = ss[3]
        l = day + " " + mon + " " + num #+ " " + hr
        extendedDateTime = day + " " + mon + " " + num + " " + hr + " " +yr
        # print("FOR INDICATOR, DATED: ",extendedDateTime)
        IaDict = { "dictDF":val, "epochForIA":convertIndicatorDateToEpoch( extendedDateTime)} #get the data for the indicator row, then the epoch date
        result.append( IaDict) #returns df/caps and the date associated    
    return result

In [47]:

def getIaDateAndGetCAPS():
    fullIaRunDown = getFullIADates() #full Ia data along with date of, as epoch as array of dict's 
    allCapsArray =[]    #a list of all dataframes for each indicator
    for i in fullIaRunDown:
        print("GETIA: for IA epoch, ", i['epochForIA'])  #access the indicator dictionary
        iaDFpre = endFrame[endFrame['time']<= i['epochForIA']].head(1000)  #rows one day before the announcement
        iaDFpost = endFrame[endFrame['time']>= i['epochForIA']].head(1000)  #rows one day before the announcement

        # print(iaDFpre.head(4))
        dict ={"IAEpoch":i['epochForIA'], "indicatorInfo":i['dictDF'], "preIACaps":iaDFpre, "postIAcaps":iaDFpost} #indicator info, name and associated rows of caps
        allCapsArray.append(dict)
    return allCapsArray    

getIaDateAndGetCAPS()

GETIA: for IA epoch,  1659358800
GETIA: for IA epoch,  1659445200
GETIA: for IA epoch,  1659531600
GETIA: for IA epoch,  1659612600
GETIA: for IA epoch,  1659699000
GETIA: for IA epoch,  1660131000
GETIA: for IA epoch,  1660131000
GETIA: for IA epoch,  1660217400
GETIA: for IA epoch,  1660309200
GETIA: for IA epoch,  1660649400
GETIA: for IA epoch,  1660735800
GETIA: for IA epoch,  1660755600
GETIA: for IA epoch,  1661340600
GETIA: for IA epoch,  1661513400
GETIA: for IA epoch,  1661513400
GETIA: for IA epoch,  1661518800
GETIA: for IA epoch,  1661864400
GETIA: for IA epoch,  1662037200
GETIA: for IA epoch,  1662118200
GETIA: for IA epoch,  1662469200
GETIA: for IA epoch,  1662550200
GETIA: for IA epoch,  1663155000
GETIA: for IA epoch,  1663241400
GETIA: for IA epoch,  1663333200
GETIA: for IA epoch,  1663673400
GETIA: for IA epoch,  1663779600
GETIA: for IA epoch,  1663779600
GETIA: for IA epoch,  1664278200
GETIA: for IA epoch,  1664537400
GETIA: for IA epoch,  1664537400
GETIA: for

[{'IAEpoch': 1659358800,
  'indicatorInfo': date         Monday, August 1, 2022
  time                        6:00 AM
  indicator     ism manufacturing pmi
  actual                         52.8
  previous                       53.0
  consensus                      52.0
  forecast                       52.2
  Name: 0, dtype: object,
  'preIACaps': Empty DataFrame
  Columns: [bc, ac, tbv, tav, time, mp, minBid]
  Index: [],
  'postIAcaps':              bc            ac        tbv        tav           time     mp  \
  0    4893271.44  1.229943e+09  629059.13  301532.22  1661520441572  23.04   
  1    4856936.23  1.229974e+09  627484.67  302885.20  1661520445856  23.02   
  2    4888973.38  1.229972e+09  628891.48  302797.25  1661520448511  23.01   
  3    4904591.43  1.229944e+09  629568.36  301568.78  1661520450877  23.03   
  4    4903009.95  1.229950e+09  629497.48  301822.89  1661520453861  23.03   
  ..          ...           ...        ...        ...            ...    ...   
  995  

: 

#### caps4IA: a dataframe of capitalization before and after an announcement

In [44]:
caps4IA = getIaDateAndGetCAPS()
print(len(caps4IA))

53


dict ={"indicatorInfo":i['dictDF'], "preIACaps":iaDFpre, "postIAcaps":iaDFpost}

currently no preIA rows appear, must inspect

In [45]:
for i in caps4IA:
    print(i)

{'indicatorInfo': date         Monday, August 1, 2022
time                        6:00 AM
indicator     ism manufacturing pmi
actual                         52.8
previous                       53.0
consensus                      52.0
forecast                       52.2
Name: 0, dtype: object, 'preIACaps': Empty DataFrame
Columns: [bc, ac, tbv, tav, time, mp, minBid]
Index: [], 'postIAcaps':              bc            ac        tbv        tav           time     mp  \
0    4893271.44  1.229943e+09  629059.13  301532.22  1661520441572  23.04   
1    4856936.23  1.229974e+09  627484.67  302885.20  1661520445856  23.02   
2    4888973.38  1.229972e+09  628891.48  302797.25  1661520448511  23.01   
3    4904591.43  1.229944e+09  629568.36  301568.78  1661520450877  23.03   
4    4903009.95  1.229950e+09  629497.48  301822.89  1661520453861  23.03   
..          ...           ...        ...        ...            ...    ...   
995  6706311.01  1.229180e+09  794230.27  289287.73  1661624134739 